In [ ]:
###===================WARNING===================###
''' 
Þegar þessi hluti er keyrður þá mun hann reyna að 
pip intsall-a numpy, prettytable og matplotlib.

Kóðin mun spyrja um leyfi y/n fyrir hvern package
ef hann er ekki nú þegar til staðar.

Ef þeir eru nú þegar í tölvunni mun kóðin skila 
"{package name} is already installed."
'''
###=============================================###

import subprocess
import sys

required_packages = [
    "numpy",
    "matplotlib",
    "linalg"
]

def install_if_missing(package):
    try:
        __import__(package)
        print(f"{package} is already installed.")
    except ImportError:
        print(f"{package} is missing.")
        choice = input(f"Do you want to install '{package}'? (y/n): ").strip().lower()

        if choice == "y":
            print(f"Installing {package}...")
            subprocess.check_call([sys.executable, "-m", "pip", "install", package])
        else:
            print(f"Skipped installing {package}.")

for pkg in required_packages:
    install_if_missing(pkg)

In [ ]:
import math
import numpy as np
from numpy import linalg as la


def bisection(f,a,b,tol):
    '''gert ráð fyrir að búið se að skilgreina f(x) fyrir utan t.d.
    def f(x):
        return(x**2-2)
    '''
    if f(a)*f(b) >= 0:
        print("Bisection method fails.")
        return None
    else:
        fa=f(a)
        while (b-a)/2>tol:
            c=(a+b)/2
            fc=f(c)
            if fc==0:break
            if fc*fa<0:
                b=c
            else:
                a=c
                fa=fc
    return((a+b)/2)

def goldensearch(a,b,tol,f):
    phi=(math.sqrt(5)-1)/2
    x1=a+(1-phi)*(b-a)
    x2=a+phi*(b-a)
    f1=f(x1)
    f2=f(x2)
    while (b-a)/2>tol:
        if f1<f2:
            b=x2
            x2=x1
            x1=a+(1-phi)*(b-a)
            f2=f1
            f1=f(x1)
        else:
            a=x1
            x1=x2
            x2=a+phi*(b-a)
            f1=f2
            f2=f(x2)
    return((a+b)/2)

def newton(x0,tol, f, Df, errors = False):
    oldx=x0+2*tol
    x=x0
    if errors:
        errors = []
        while abs(oldx-x)>tol:
            oldx=x
            x=x-f(x)/Df(x)
            errors.append(abs(oldx-x))
        return x, errors
    else:
        while abs(oldx-x)>tol:
            oldx=x
            x=x-f(x)/Df(x)
        return x
    
def newtonmult(x0,tol,F,DF):
    '''
    x0 er vigur i R^n skilgreindur t.d. sem
    x0=np.array([1,2,3])
    gert ráð fyrir að F(x) og Jacobi fylki DF(x) séu skilgreind annars staðar
    '''
    x=x0
    oldx=x+2*tol
    while la.norm(x-oldx,np.inf)>tol:
        oldx=x
        s=-la.solve(DF(x),F(x))
        x=x+s
    return(x)

def poisuilles(x,G,p1,p0):

    poisuilles_gildi = [
    (G,   p1,  x[0]),       # q_1A 
    (G,   x[0], x[1]),      # q_AB 
    (G,   x[0], x[2]),      # q_AC 
    (G,   x[1], x[3]),      # q_BD 
    (G,   x[2], x[3]),      # q_CD 
    ((2)*G,   x[2], x[4]),  # q_CE 
    ((2/3)*G, x[3], x[4]),  # q_DE 
    (G,   x[4], p0)         # q_E0 
    ]

    return [g * (i - j) for g, i, j in poisuilles_gildi] #[q_1A, q_AB, q_AC, q_BD, q_CD, q_CE, q_DE, q_E0]

def q_E0(A,B,C,w,t):

    q = A*np.cos(w*t) + B*np.sin(w*t) + C

    return q

def linal_solver(p1, G, Q_b):
    
    A = np.matrix('3 -1 -1 0 0; 1 -2 0 1 0; 1 0 -4 1 2; 0 3 3 -8 2; 0 0 6 2 -11')
    B = np.matrix(f'{p1};{-Q_b/G}; 0; 0; 0')

    # Nota linalg til að leysa fyrir x
    x = la.solve(A,B)
    
    return x

def modifyed_bisection(target_q, G, p0, Q_b, tol=1e-6, max_iter=100):

    # Algorithm bounds
    low = 0
    high = 6e6   # þrýstingur í dæmi 1 var 4.2e6, svo þetta dugir

    for _ in range(max_iter):
        mid = 0.5*(low + high)

        # leysa línulega kerfið fyrir mid gildi
        x = linal_solver(mid, G, Q_b)
        q = poisuilles(x, G, mid, p0)[-1]  # q_E0

        if abs(q - target_q) < tol:
            return mid

        # helmingun
        if q > target_q:
            high = mid
        else:
            low = mid

    return mid  # ef við náum ekki alveg nákvæmni

def darcy_weisback(K,q_ij,p_j):
    p_i = K*q_ij*abs(q_ij)+p_j
    return p_i

def compute_pressures(sol, K, p0):
    # name, multiplier for K, sol_index, parent_name
    steps = [
        ("PE", 1.0, -1, "P0"),
        ("PD", 1.5, -2, "PE"),
        ("PC", 1.0, 4,  "PD"),
        ("PB", 1.0, 3,  "PD"),
        ("PA", 1.0, 1,  "PB"),
        ("P1", 1.0, 0,  "PA"),
    ]

    P = {"P0": p0}

    for name, k_mult, sol_idx, parent in steps:
        P[name] = darcy_weisback(k_mult * K, sol[sol_idx], P[parent])

    return P

In [ ]:
import numpy as np
from numpy import linalg as la
import math as m
from functions import poisuilles

# Fastar #
v = 1*10**(-3)
r = 5*10**(-2)
L = 100
G = (m.pi*(r**4))/(8*v*L)
p1 = 4.2*10**(6)
p0 = 0
Q_b = 7

# Skilgreining á fyljunum a og b fyrir ax = b
A = np.matrix('3 -1 -1 0 0; 1 -2 0 1 0; 1 0 -4 1 2; 0 3 3 -8 2; 0 0 6 2 -11')
B = np.matrix(f'{p1};{-Q_b/G}; 0; 0; 0')

# Nota linalg til að leysa fyrir x
x = la.solve(A,B)

# Reikna flæði í röri milli punkta með poisuilles jöfnunni
q = poisuilles(x,G,p1,p0)

# Print
print()
print('Lausn línulegu jöfnunnar:')
print('P_A               P_B               P_C               P_D               P_E')
print('  '.join(map(str, x)).replace("[[","").replace("]]",""))
print()
print('Flæði gildi í rörum milli nóða i og j -> q_ij')
print('q_1A         q_AB         q_AC         q_BD          q_CD        q_CE        q_DE         q_E0')
print('  '.join(map(str, q)).replace("[[","").replace("]]",""))
print()

In [ ]:
import functions as f
import numpy as np

import matplotlib.pyplot as plt

w = 2*np.pi/24 #Gögn úr dæmi

t = [1, 3, 5, 7, 9, 11, 13, 15, 17, 19, 21, 23] #Gögn úr dæmi
b = [12.0, 14.0, 13.9, 12.3, 11.7, 8.9, 6.3, 6.5, 6.1, 5.8, 6.6, 9.3] #Gögn úr dæmi
a = np.matrix([[np.cos(w*x), np.sin(w*x), 1] for x in t]) #Fylkið A skv. aðverð minnstu kvarðata
aT = np.transpose(a) #Transpose af A
aTa = np.dot(aT,a) #Transpose af A margfaldað við A
aTaI = np.linalg.inv(aTa) #Adnhverfa af margefeldi Transpose A og A
x = np.dot(np.dot(aTaI, aT), b) #Andhverfan margfaldað við transpose A og b skv. setningu gefur x

print("A:", x[0,0])
print("B:", x[0,1])
print("C:", x[0,2])


# plt.plot(t,b,'o')
# plt.plot(t,[f.qE0(i,x[0,0],x[0,1],x[0,2]) for i in t])
# plt.show() #It's beautiful

In [ ]:
import numpy as np
import math as m
from functions import q_E0
from functions import modifyed_bisection

v = 1*10**(-3)
r = 5*10**(-2)
L = 100
G = (m.pi*(r**4))/(8*v*L)
Q_b = 7
p0 = 0

A_val = 1.3209447623073862
B_val = 4.015670949654589
C_val = 9.45


w = 2*np.pi/24

# 100 tímapunktar
t_values = np.linspace(0, 24, 100)

p1_values = []

for t in t_values:
    target_q = q_E0(A_val, B_val, C_val, w, t)
    p1 = modifyed_bisection(target_q, G, p0, Q_b)
    p1_values.append(p1)

print('p1 gildi:')
print('  t       p1 gildi')
for i in range(len(p1_values)):
    print(f'{t_values[i]:.2f} h  {p1_values[i]:.3e} Pa')

In [ ]:
import numpy as np
from numpy import linalg as LA
from functions import newtonmult
from functions import compute_pressures

def F(x):
    q1A, qAB, qAC, qBD, qCD, qCE, qDE, qE0 = x
    Fx = np.zeros(8)
    Fx[0] = q1A - qAB - qAC
    Fx[1] = qAB - qBD + QB
    Fx[2] = qAC - qCD - qCE
    Fx[3] = qBD + qCD - qDE
    Fx[4] = qCE + qDE - qE0
    Fx[5] = (qAB * abs(qAB) - qAC * abs(qAC) + qBD * abs(qBD) - qCD * abs(qCD))
    Fx[6] = (qCD * abs(qCD) - 0.5 * qCE * abs(qCE)) + 1.5 * qDE * abs(qDE)
    Fx[7] = eih + q1A * abs(q1A) + qAC * abs(qAC) + 0.5 * qCE * abs(qCE) + qE0 * abs(qE0)
    
    return Fx

def DF(x):
    q1A, qAB, qAC, qBD, qCD, qCE, qDE, qE0 = x

    J = np.zeros((8, 8))

    J[0, 0] =  1.0
    J[0, 1] = -1.0
    J[0, 2] = -1.0
    J[1, 1] =  1.0
    J[1, 3] = -1.0
    J[2, 2] =  1.0
    J[2, 4] = -1.0
    J[2, 5] = -1.0
    J[3, 3] =  1.0
    J[3, 4] =  1.0
    J[3, 6] = -1.0
    J[4, 5] =  1.0
    J[4, 6] =  1.0
    J[4, 7] = -1.0
    J[5, 1] =  2.0 * abs(qAB)
    J[5, 2] = -2.0 * abs(qAC)
    J[5, 3] =  2.0 * abs(qBD)
    J[5, 4] = -2.0 * abs(qCD)
    J[6, 4] =  2.0 * abs(qCD)
    J[6, 5] = -1.0 * abs(qCE)
    J[6, 6] =  3.0 * abs(qDE)
    J[7, 0] = 2.0 * abs(q1A)
    J[7, 2] = 2.0 * abs(qAC)
    J[7, 5] = 1.0 * abs(qCE)
    J[7, 7] = 2.0 * abs(qE0)

    return J

QB = 7.0
p0 = 0.0
p1 = 4.2*10**6
K  = 1.617897*10**8
eih = (p0 - p1) / K

x = ["q1A", "qAB", "qAC", "qBD", "qCD", "qCE", "qDE", "qE0"]
x0 = np.array([1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0])
tol = 1*10**-8
sol = newtonmult(x0, tol, F, DF)
for i,j in enumerate(x):
    print(j,": ", sol[i], sep="")

#pressure
P = compute_pressures(sol, K, p0)
for name in ["P1","PA","PB","PC","PD","PE","P0"]:
    print(f"{name}: {P[name]}")